In [18]:
import pandas as pd
import random
import time
import json
from random_user_agent.params import SoftwareName, OperatingSystem
from random_user_agent.user_agent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
# from eunjeon import Mecab
import unicodedata
import pyodbc

class IG_Bot:
    '''creating a Bot that acts like me.'''
    
    def __init__(self, username, password):
        software_names= [SoftwareName.CHROME.value]
        operating_systems=[OperatingSystem.WINDOWS.value]
        
        #Randomized User ID
        user_agent_rotator= UserAgent(software_name = software_names,
                                     operating_systems=operating_systems,
                                     limit=100)
        user_agent= user_agent_rotator.get_random_user_agent()
        chrome_options=Options()
#         chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sendbox")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument(f'user-agent={user_agent}')
        path='c:\python_temp\chromedriver.exe'
        
        cookies_file_path='c:\IG\cookies.json'
        cookie_websites=['https://instagram.com']
        self.cookies_file_path= cookies_file_path
        self.cookie_websites= cookie_websites
        self.driver=webdriver.Chrome(chrome_options=chrome_options,executable_path=path)
        
        self.driver.get('https://instagram.com')
        
        time.sleep(1.5)

        try:
            cookies=json.load(open(self.cookies_file_path,'rb'))
            for website in self.cookie_websites:
                for cookie in cookies:
                    self.driver.add_cookie(cookie)
                self.driver.refresh()
        except Exception as e:
            print(str(e))
    def loading(self,xpath):
        '''Wait Loading Until presence'''
        WebDriverWait(self.driver,10).until(EC.presence_of_element_located((By.XPATH,xpath)))
    
    def save_cookies(self):
        '''Save cookie'''
        json.dump(self.driver.get_cookies(),open('c:\IG\cookies.json','w'))
    
    def load_cookies(self):
        '''Load cookies into browser'''
        cookies= json.load(open('cookies.json','rb'))
        for cookie in cookies:
            self.driver.add_cookie(cookie)
    
    def login(self):
        '''Login to Instagram'''
        try:
            self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(username)
            self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(password)
            self.loading('//*[@id="loginForm"]/div/div[3]')
            time.sleep(0.5)
            self.driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
            self.loading('//*[@id="react-root"]/section/main/div/div/div/div/button')
            self.driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
            self.loading('/html/body/div[5]/div/div/div/div[3]/button[2]')
            self.driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
        except:
            self.loading('/html/body/div[5]/div/div/div/div[3]/button[2]')
            self.driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
class Crawler(IG_Bot):
    def run(self):
        self.driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
        query=input('검색어를 입력하시오. > ')
        query='#'+query
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(query)
        self.driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(Keys.ENTER)
    def crawl(self):
        insta_dict = {'img':[],
                        'id':[],
                      'date': [],
                      'like': [],
                      'text': [],
                      'hashtag':[]}
        ## 이미지 수집

        try:
            srcs=self.driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div/div/div/div/video')
            src=srcs.get_attribute('poster')
            insta_dirct['img'].append(src)
        except:
            src=self.driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div[1]/div[2]/div/div/div/ul/li/div/div/div/div/img')
            src=src.get_attribute('src')
            insta_dict['img'].append(src)
        ## id 정보 수집
        try:
            info_id = self.driver.find_element_by_css_selector('h2._6lAjh').text
            insta_dict['id'].append(info_id)
        except:
            info_id = self.driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
            insta_dict['id'].append(info_id)
        
        print()


        ## 시간정보 수집 
        upload_time=self.driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/div[2]/a/time')
        print(upload_time.get_attribute('title'))
        time_raw = self.driver.find_element_by_css_selector('time.FH9sR.Nzb55')
        time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
        insta_dict['date'].append(time_info)
        print(insta_dict['date'])
        print()

        ## like 정보 수집
        try:
            check=self.driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/span/span')

        except:
            try:
                like=self.driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/div/a/span')
                insta_dict['like'].append(like)

            except:
                id=[]
                self.driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/div[2]/a[2]').click()
                while True:
                    last_height =self.driver.find_element_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a').location

                    c=self.driver.find_elements_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a')

                    for i in c:  
                        if i.text not in id:
                            id.append(i.text)     
                        else:
                            pass
                    self.driver.find_element_by_css_selector('div._1XyCr').click()
                    self.driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                    self.driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                    self.driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                    time.sleep(1)
                    new_height =self.driver.find_element_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a').location

                    if last_height==new_height:
                        self.driver.find_element_by_xpath('/html/body/div[7]/div/div/div[1]/div/div[2]/button').click()
                        break

                like=len(id)
                insta_dict['like'].append(like)

        ##text 정보수집
        raw_info = self.driver.find_element_by_css_selector('div.C4VMK').text.split()
        raw_info = raw_info[1:len(raw_info)-1]
        text=[]
        for i in raw_info:
            if '#' in i:
                pass
            else:
                text.append(i)
        clean_text = ' '.join(text)
        text.append(clean_text)
        insta_dict['text'].append(text)



        ##hashtag 수집
        raw_tags = self.driver.find_elements_by_css_selector('a.xil3i')
        hash_tag = []
        for i in raw_tags:
            hash_tag.append(i.text)
        insta_dict['hashtag'].append(hash_tag)
            
if __name__=='__main__':
    "Main (runs through classes)"
    data=pd.read_csv('c:\IG\secret.txt',header=None)
    username=data.iloc[0][0]
    password=data.iloc[1][0]
    
    '''Check login to instagram'''
#     bot=IG_Bot(username,password)
#     bot.login()
#     bot.save_cookies()

'''Crawler'''
bot=Crawler(username,password)

<ipython-input-18-3d95a56cf44d>:42: DeprecationWarning: use options instead of chrome_options
  self.driver=webdriver.Chrome(chrome_options=chrome_options,executable_path=path)


In [20]:
from selenium import webdriver
driver=webdriver.Chrome('c:\python_temp\chromedriver.exe')
driver.get('https://instagram.com')

In [25]:
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys('#호호')


In [27]:
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(Keys.ENTER)

In [5]:
bot.crawl()

['hello_storystudio']

2021년 9월 28일
[Timestamp('2021-09-28 00:00:00+0000', tz='UTC')]



In [19]:
bot.run()

검색어를 입력하시오. > 마우스


In [3]:
# driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div[1]/div[2]').click()
import time
list=[]
while True:
    try:
        srcs=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div/div[1]/img')
        list.append(srcs.get_attribute('src'))
    except:
        pass
    try:
        try:    
            srcs=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div[1]/div[2]/div/div/div/ul/li/div/div/div/div/div/img')
            time.sleep(1)
            list.append(srcs.get_attribute('src'))
        except:
            pass
        srcs=driver.find_elements_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div[1]/div[2]/div/div/div/ul/li/div/div/div/div/div/img')
        print(srcs.get_attribute('src'))
    except:
        pass
    srcs=driver.find_elements_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div[1]/div[2]/div/div/div/ul/li/div/div/div/div[1]/img')
    time.sleep(2)
    for i in srcs:
        try:
            if i.get_attribute('src')==' ':
                pass
            if i.get_attribute('src') not in list:
                print(i.get_attribute('src'))
                list.append(i.get_attribute('src'))
        except Exception as e:
            pass

        else:
            pass
    try:
        driver.find_element_by_class_name('_6CZji').click()
        time.sleep(0.5)
    except:
        break

print(list)
len(list)


https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/244980205_585032922837382_862819861627148663_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=7qH8kjTnBWkAX-XijH9&tn=XWqKIzVDEZ5cxi9m&edm=AP_V10EBAAAA&ccb=7-4&oh=e321596f4c99739edb56d369ae77d4f4&oe=616BFB79&_nc_sid=4f375e
['https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/245073152_190779459863442_5808265287069426276_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=106&_nc_ohc=XK3JrYtGl8MAX9rZLsO&edm=AP_V10EBAAAA&ccb=7-4&oh=47b68bc84f33e53a3782e3e98af4d389&oe=616B6BCF&_nc_sid=4f375e', 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/244980205_585032922837382_862819861627148663_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=7qH8kjTnBWkAX-XijH9&tn=XWqKIzVDEZ5cxi9m&edm=AP_V10EBAAAA&ccb=7-4&oh=e321596f4c99739edb56d369ae77d4f4&oe=616BFB79&_nc_sid=4f375e', 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/245073152_1907

3

In [136]:
import pandas as pd
from selenium.webdriver.common.keys import Keys
def crawl(self):
    insta_dict = {'img':[],
                    'id':[],
                  'date': [],
                  'like': [],
                  'text': [],
                  'hashtag':[]}
    ## 이미지 수집

    try:
        srcs=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div/div/div/div/video')
        src=srcs.get_attribute('poster')
        insta_dirct['img'].append(src)
    except:
        src=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[1]/div/div[1]/div[2]/div/div/div/ul/li/div/div/div/div/img')
        src=src.get_attribute('src')
        insta_dict['img'].append(src)
    ## id 정보 수집
    try:
        info_id = driver.find_element_by_css_selector('h2._6lAjh').text
        insta_dict['id'].append(info_id)
    except:
        info_id = driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
        insta_dict['id'].append(info_id)
    print(insta_dict['id'])
    print()


    ## 시간정보 수집 
    upload_time=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/div[2]/a/time')
    print(upload_time.get_attribute('title'))
    time_raw = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
    time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
    insta_dict['date'].append(time_info)
    print(insta_dict['date'])
    print()

    ## like 정보 수집
    try:
        check=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/span/span')
        
    except:
        try:
            like=driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/div/a/span')
            insta_dict['like'].append(like)
            
        except:
            id=[]
            driver.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/div/div[2]/div/div[2]/section[2]/div/div[2]/a[2]').click()
            while True:
                last_height =driver.find_element_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a').location

                c=driver.find_elements_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a')

                for i in c:  
                    if i.text not in id:
                        id.append(i.text)     
                    else:
                        pass
                driver.find_element_by_css_selector('div._1XyCr').click()
                driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
                time.sleep(1)
                new_height =driver.find_element_by_xpath('/html/body/div[7]/div/div/div[3]/div/div/div/div/div/div/span/a').location

                if last_height==new_height:
                    driver.find_element_by_xpath('/html/body/div[7]/div/div/div[1]/div/div[2]/button').click()
                    break

            like=len(id)
            insta_dict['like'].append(like)

    ##text 정보수집
    raw_info = driver.find_element_by_css_selector('div.C4VMK').text.split()
    raw_info = raw_info[1:len(raw_info)-1]
    text=[]
    for i in raw_info:
        if '#' in i:
            pass
        else:
            text.append(i)
    clean_text = ' '.join(text)
    text.append(clean_text)
    insta_dict['text'].append(text)



    ##hashtag 수집
    raw_tags = driver.find_elements_by_css_selector('a.xil3i')
    hash_tag = []
    for i in raw_tags:
        hash_tag.append(i.text)
    insta_dict['hashtag'].append(hash_tag)

https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/239218352_335498761650611_1251356593086644312_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=G78EYJ1SFPMAX_B6Pku&edm=AP_V10EBAAAA&ccb=7-4&oh=76b23c271d1ea87146c2781573ac94fe&oe=616D2144&_nc_sid=4f375e
['hello_storylibrary']

2021년 8월 20일
[Timestamp('2021-08-20 00:00:00+0000', tz='UTC')]

26

["[스토리라이브러리만의 특별한 영화 추천, 스라의 빛✨소개] 🎬 너무 좋아서 벽치다가 무너져서 🎬 옆집 만날 장르?! 🎬 말이 필요없는 망작?! 🎬 꿈에 나올것 같았던 영화?! 🎬 간단하게 보기 좋은 영화?! 🎬 꿈인지 현실인지 헷갈리는 영화?! 스라러 재영이 만들어준 영화 추천 큐레이션 게시판, '스라의 빛' 반응이 핫해요! 이미 있는 영화 장르를 제외하고 새로운 장르와 장르에 어울리는 영화를 추천하는 게시판이에요. 게시판이 생긴지 한달도 되지 않아서 어느새 빼곡히 추천 장르, 영화로 가득찼어요. 하루 하루 스라러들이 슬쩍 남겨주고 간 흔적을 읽어보는 재미가 쏠쏠해요. 새로운 영화를 살펴보며 낯선 영감을 만나고 싶다면 스라로 놀러오세요! 오늘도 스토리라이브러리 문 활-짝! 저녁 6시까지 문을 열어둘께요. 9월부터는 수, 목, 금요일은 오후 5시부터 저녁 9시까지, 주말엔 오후 12시부터 6시까지 문을 열어요! 놀러오고 싶다면 프로필 링크를 클릭👆"]

['#스토리라이브러리', '#스라', '#스라러', '#스라의빛', '#영화', '#영화추천', '#장르', '#장르추천', '#1219세전용작업실', '#혜화역2번출구']
